In [18]:
import requests
import datetime
import statistics
import os
import math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [19]:
# --- EODHD API Configuration ---
OPTIONS_API_BASE_URL = "https://eodhd.com/api/mp/unicornbay/options"
EQUITY_API_BASE_URL = "https://eodhd.com/api" 

# --- Get API Key (Recommended: Use environment variable) ---
# Make sure to set EODHD_API_TOKEN environment variable
# Or replace os.getenv(...) with your actual key string
API_TOKEN = '67fb3d6f50c489.92544905'

# --- Analysis Parameters ---
TICKER = 'QQQ'       # Ticker symbol to analyze
MIN_DTE = 25         # Minimum Days To Expiration
MAX_DTE = 35         # Maximum Days To Expiration (adjust as needed, e.g., 30, 35, 45)
LOOKBACK_DAYS = 365  # Lookback period in days for historical IV
ATM_PERCENT = 10     # Percentage range (+/-) around stock price to consider ATM

# --- Check API Key ---
if not API_TOKEN or API_TOKEN == 'YOUR_API_KEY':
    print("🛑 ERROR: API Token not configured.")
    print("Please set the EODHD_API_TOKEN environment variable or replace 'YOUR_API_KEY' in the script.")
    # You might want to raise an error here or prevent further execution in a real application
    # raise ValueError("API Token not configured.")
else:
    print("✅ API Token found.")

✅ API Token found.


In [20]:
def get_eodhd_data(base_url, endpoint_path, params):
    """Fetches data from a specified EODHD API base URL."""
    if not API_TOKEN or API_TOKEN == 'YOUR_API_KEY':
        print("API Token is missing!")
        return None # Return None if key is missing

    url = f"{base_url}/{endpoint_path}"
    params['api_token'] = API_TOKEN
    params['fmt'] = 'json' # Ensure response is JSON

    response = None # Initialize response to None
    try:
        # print(f"DEBUG: Calling URL: {url} with params: {params}") # Uncomment for debugging API calls
        response = requests.get(url, params=params)
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        # print(f"DEBUG: Received data type: {type(data)}") # Debugging
        return data
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching data from EODHD API ({url}): {e}")
        if response is not None:
            print(f"Response status code: {response.status_code}")
            print(f"Response text: {response.text[:500]}...") # Print first 500 chars
        return None
    except ValueError as e: # Catches JSONDecodeError if response isn't JSON
        print(f"❌ Error decoding JSON response from {url}: {e}")
        if response is not None:
            print(f"Response text: {response.text[:500]}...")
        return None

In [21]:
def get_stock_price(ticker):
    """Gets the latest closing price for the ticker."""
    print(f"Fetching latest stock price for {ticker}...")
    # Use the EOD endpoint for the equity
    # Append ".US" exchange code - adjust if needed for other exchanges
    # QQQ is typically on NASDAQ, so .US should work
    endpoint = f"eod/{ticker.upper()}.US" 
    params = {'order': 'd', 'limit': 1} # Get descending order, limit 1 (latest day)

    data = get_eodhd_data(EQUITY_API_BASE_URL, endpoint, params)

    if data and isinstance(data, list) and len(data) > 0:
        latest_data = data[0]
        price = None
        if 'adjusted_close' in latest_data:
            price = latest_data['adjusted_close']
        elif 'close' in latest_data:
             price = latest_data['close'] # Fallback to close if adjusted isn't there
        
        if price is not None:
             print(f"   -> Latest Price ({latest_data.get('date', 'N/A')}): {price:.2f}")
             return price
             
    print(f"⚠️ Warning: Could not retrieve latest stock price for {ticker}.")
    return None

In [22]:
def calculate_dte(exp_date_str):
    """Calculates Days To Expiration from a YYYY-MM-DD string."""
    try:
        exp_date = datetime.datetime.strptime(exp_date_str, '%Y-%m-%d').date()
        today = datetime.date.today()
        dte = (exp_date - today).days
        return dte
    except (ValueError, TypeError):
        return None

In [23]:
def calculate_iv_rank(ticker, min_dte=25, max_dte=30, lookback_days=365, atm_percent=10):
    """
    Calculates IV Rank using ATM options for current IV and history basis.
    Returns a dictionary with results and historical data points.
    """
    print(f"\n🔄 Starting IV Rank Calculation for {ticker}...")
    print(f"Parameters: DTE={min_dte}-{max_dte}, Lookback={lookback_days} days, ATM=+/-{atm_percent}%")

    # --- 1. Get Current Stock Price ---
    stock_price = get_stock_price(ticker)
    if stock_price is None:
        print(f"❌ Error: Cannot proceed without stock price for {ticker}.")
        return None

    # --- 2. Find Current Options in DTE range ---
    print(f"\n🔍 Step 2: Finding options expiring in {min_dte}-{max_dte} days...")
    today = datetime.date.today()
    target_exp_date_start = today + datetime.timedelta(days=min_dte)
    target_exp_date_end = today + datetime.timedelta(days=max_dte)

    contracts_params = {
        'filter[underlying_symbol]': ticker,
        'filter[exp_date_from]': target_exp_date_start.strftime('%Y-%m-%d'),
        'filter[exp_date_to]': target_exp_date_end.strftime('%Y-%m-%d'),
        'fields[options-contracts]': 'contract,exp_date,volatility,strike,type',
        'limit': 5000 # Increase limit if needed for very liquid tickers
    }

    contracts_data = get_eodhd_data(OPTIONS_API_BASE_URL, 'contracts', contracts_params)

    if not contracts_data or 'data' not in contracts_data or not contracts_data['data']:
        print(f"❌ Error: No options data found for {ticker} in the {min_dte}-{max_dte} DTE range.")
        return {"error": "No options found in DTE range", "ticker": ticker}

    # --- 3. Filter for ATM Contracts & Calculate Current ATM IV ---
    print(f"\n📊 Step 3: Filtering for ATM (+/- {atm_percent}%) contracts & calculating current average ATM IV...")
    atm_ivs = []
    atm_contracts = []
    min_strike_diff = float('inf')
    closest_atm_contract = None

    for contract in contracts_data['data']:
        attributes = contract.get('attributes', {})
        exp_date_str = attributes.get('exp_date')
        volatility = attributes.get('volatility')
        strike = attributes.get('strike')
        contract_id = attributes.get('contract')

        if exp_date_str and contract_id and strike is not None:
            dte = calculate_dte(exp_date_str)
            # Check DTE range
            if dte is not None and min_dte <= dte <= max_dte:
                try:
                    strike_float = float(strike)
                    # Check ATM range
                    strike_diff = abs(strike_float - stock_price)
                    if strike_diff <= stock_price * (atm_percent / 100.0):
                        if volatility is not None:
                            try:
                                iv = float(volatility)
                                if iv > 0.01: # Basic sanity check for positive, non-negligible IV
                                     atm_ivs.append(iv)
                                     atm_contracts.append(contract)
                                     # Track the contract closest to the stock price
                                     if strike_diff < min_strike_diff:
                                         min_strike_diff = strike_diff
                                         closest_atm_contract = contract
                            except (ValueError, TypeError): pass # Silently skip invalid volatility
                except (ValueError, TypeError): pass # Silently skip invalid strike

    if not atm_ivs:
        print(f"❌ Error: No valid ATM options found for {ticker} within +/- {atm_percent}% in the {min_dte}-{max_dte} DTE range.")
        return {"error": "No ATM options found", "ticker": ticker, "stock_price": stock_price}

    if not closest_atm_contract:
        print(f"❌ Error: Could not identify a closest ATM contract for historical lookup.")
        # Fallback: Maybe use the first ATM contract found? Or return error.
        closest_atm_contract = atm_contracts[0] # Use first found as fallback
        print("⚠️ Warning: Using the first found ATM contract for history as closest couldn't be determined.")
        # return {"error": "No closest ATM contract found", "ticker": ticker, "stock_price": stock_price}


    current_avg_atm_iv = statistics.mean(atm_ivs)
    print(f"   Found {len(atm_ivs)} ATM options with valid IV.")
    print(f"   Current Average ATM IV ({min_dte}-{max_dte} DTE): {current_avg_atm_iv:.4f}")


    # --- 4. Get Historical IV for the Representative ATM Contract ---
    print("\n📜 Step 4: Fetching historical IV data (using closest ATM contract)...")
    representative_contract_attrs = closest_atm_contract['attributes']
    rep_contract_id = representative_contract_attrs['contract']
    print(f"   Using closest ATM contract {rep_contract_id} (Exp: {representative_contract_attrs['exp_date']}, Strike: {representative_contract_attrs['strike']}, Type: {representative_contract_attrs['type']}) for history.")

    hist_start_date = today - datetime.timedelta(days=lookback_days)
    hist_end_date = today # Go up to yesterday/today

    eod_params = {
        'filter[contract]': rep_contract_id,
        'from': hist_start_date.strftime('%Y-%m-%d'),
        'to': hist_end_date.strftime('%Y-%m-%d'),
        'fields[options-eod]': 'date,volatility',
        'limit': lookback_days + 50 # Ensure we get enough data points if some are missing
    }

    historical_data_raw = get_eodhd_data(OPTIONS_API_BASE_URL, 'eod', eod_params)
    
    historical_data_points = [] # Store tuples of (date, iv)
    
    if not historical_data_raw or 'data' not in historical_data_raw or not historical_data_raw['data']:
        print(f"⚠️ Warning: No historical EOD data found for representative contract {rep_contract_id}.")
        historical_ivs = []
    else:
         historical_ivs = []
         for day_data in historical_data_raw['data']:
             attributes = day_data.get('attributes', {})
             volatility = attributes.get('volatility')
             date_str = attributes.get('date') # EOD endpoint uses 'date' field
             if volatility is not None and date_str:
                 try:
                     iv = float(volatility)
                     if iv > 0.01: # Only consider valid positive IVs
                         historical_ivs.append(iv)
                         hist_date = datetime.datetime.strptime(date_str, '%Y-%m-%d').date()
                         historical_data_points.append((hist_date, iv))
                 except (ValueError, TypeError): pass # Ignore invalid values

    # Sort historical data points by date for plotting
    historical_data_points.sort(key=lambda x: x[0])

    # --- 5. Determine Historical High/Low IV ---
    print("\n📉 Step 5: Determining historical IV range...")
    hist_low_iv = None
    hist_high_iv = None
    iv_rank = None
    
    if not historical_ivs:
        print("   No valid historical volatility values found. Cannot determine range or rank.")
    else:
        if len(historical_ivs) < lookback_days * 0.1: # Warn if data seems very sparse
            print(f"   ⚠️ Warning: Found only {len(historical_ivs)} valid historical IV points in {lookback_days} days for {rep_contract_id}.")
        
        hist_low_iv = min(historical_ivs)
        hist_high_iv = max(historical_ivs)
        print(f"   Historical IV Low ({lookback_days}-day): {hist_low_iv:.4f}")
        print(f"   Historical IV High ({lookback_days}-day): {hist_high_iv:.4f}")

        # --- 6. Calculate IV Rank ---
        print("\n🧮 Step 6: Calculating IV Rank...")
        iv_range = hist_high_iv - hist_low_iv

        if iv_range < 0.0001: # Use a small threshold instead of exact zero
            iv_rank = 50.0 if abs(current_avg_atm_iv - hist_low_iv) < 0.0001 else (0.0 if current_avg_atm_iv < hist_low_iv else 100.0)
            print(f"   ⚠️ Warning: Historical IV range near zero. Setting IV Rank based on current vs historical level ({iv_rank:.1f}).")
        else:
            iv_rank = ((current_avg_atm_iv - hist_low_iv) / iv_range) * 100.0
            iv_rank = max(0.0, min(100.0, iv_rank)) # Clamp between 0 and 100
            print(f"   Calculated IV Rank: {iv_rank:.2f}")

    # --- 7. Prepare Result ---
    result = {
        "ticker": ticker,
        "stock_price": stock_price,
        "current_avg_atm_iv": current_avg_atm_iv,
        "hist_low_iv": hist_low_iv,
        "hist_high_iv": hist_high_iv,
        "iv_rank": iv_rank,
        "historical_data_points": historical_data_points, # List of (date, iv) tuples
        "representative_contract": rep_contract_id,
        "dte_range": f"{min_dte}-{max_dte}",
        "lookback_days": lookback_days
    }
    
    print("\n✅ Calculation Complete.")
    return result

In [24]:
# Check if API Token is valid before proceeding
if API_TOKEN and API_TOKEN != 'YOUR_API_KEY':
    # Call the function with parameters defined in the Configuration cell
    iv_rank_result = calculate_iv_rank(
        ticker=TICKER,
        min_dte=MIN_DTE,
        max_dte=MAX_DTE,
        lookback_days=LOOKBACK_DAYS,
        atm_percent=ATM_PERCENT
    )

    # Display the final calculated values
    print("\n" + "="*30)
    print(f"    FINAL IV RANK SUMMARY for {TICKER}")
    print("="*30)
    if iv_rank_result and 'error' not in iv_rank_result:
        print(f"Ticker:                 {iv_rank_result['ticker']}")
        print(f"Current Stock Price:    {iv_rank_result['stock_price']:.2f}")
        print(f"DTE Range Analyzed:     {iv_rank_result['dte_range']} days")
        print(f"Current Avg ATM IV:     {iv_rank_result['current_avg_atm_iv']:.4f}")
        if iv_rank_result['hist_low_iv'] is not None:
            print(f"{iv_rank_result['lookback_days']}-Day IV Low:         {iv_rank_result['hist_low_iv']:.4f}")
            print(f"{iv_rank_result['lookback_days']}-Day IV High:        {iv_rank_result['hist_high_iv']:.4f}")
            print(f"IV Rank:                {iv_rank_result['iv_rank']:.2f}")
        else:
            print(f"{iv_rank_result['lookback_days']}-Day IV Low:         N/A")
            print(f"{iv_rank_result['lookback_days']}-Day IV High:        N/A")
            print(f"IV Rank:                N/A (Insufficient historical data)")
        print(f"History Based On:       {iv_rank_result['representative_contract']}")

    elif iv_rank_result:
         print(f"Calculation failed: {iv_rank_result.get('error', 'Unknown error')}")
    else:
        print("Calculation failed, no result returned.")
    print("="*30)

else:
    print("\n🛑 Cannot execute calculation: API Token is missing or invalid.")


🔄 Starting IV Rank Calculation for QQQ...
Parameters: DTE=25-35, Lookback=365 days, ATM=+/-10%
Fetching latest stock price for QQQ...
   -> Latest Price (2025-04-16): 444.18

🔍 Step 2: Finding options expiring in 25-35 days...

📊 Step 3: Filtering for ATM (+/- 10%) contracts & calculating current average ATM IV...
   Found 36 ATM options with valid IV.
   Current Average ATM IV (25-35 DTE): 0.3292

📜 Step 4: Fetching historical IV data (using closest ATM contract)...
   Using closest ATM contract QQQ250516C00445000 (Exp: 2025-05-16, Strike: 445, Type: call) for history.

📉 Step 5: Determining historical IV range...
   No valid historical volatility values found. Cannot determine range or rank.

✅ Calculation Complete.

    FINAL IV RANK SUMMARY for QQQ
Ticker:                 QQQ
Current Stock Price:    444.18
DTE Range Analyzed:     25-35 days
Current Avg ATM IV:     0.3292
365-Day IV Low:         N/A
365-Day IV High:        N/A
IV Rank:                N/A (Insufficient historical da